In [2]:
import numpy as np

# Magic Cake Problem
**Nick Brown**
**GSE 512**  

You have this utility function:
$$E[\sum_{t=0}^{\infty} \beta^t u(c_t)]$$

and a nice genie has given you a magic cake. The cakes size at time 0 is $y_0 z_0$, a known number. You get to eat a piece of any size from that initial cake.  The genie waves her magic wand over night, and she shocks the cake with a multiplicative random variable $z_t$ whose logarithm is independently and identically distributed $N \sim (\mu, \sigma^2)$. Your goal is to maximize expected utility.

Describe this problem as a dynamic program:

What is the state?  
> The state space is denoted as $X$. In this specific scenario, the state space would be the size of the cake that we start out with ($y_0 z_0$).

What is the control?  
> The control is denoted as $U$. In this scenario, $U$ would represent the amount of cake we choose to eat subject to the size of the cake. We can't eat more than the whole cake.

What is the law of motion?  
> The law of motion is some rule for transition into the next period. So for this problem the law of motion is the random multiplicative shock that the genie does to the remaining cake each day ($z_t \times (y_0 - c_0)$).

What is the reward? 
> The reward is denoted as r, and is defined as $r: X \times U \rightarrow \mathbb{R}$. The interpretation of $r(x,u)$ is the reward in state $x \in X$ if I pick control $u \in U$. So if I have 10 pounds of cake, and choose to eat a quarter of it, $r$ would represent the reward for that choise.

What is the discount factor?  
> The discount factor represents how much I discount forgoing current felicity in favor of future felicity. The discount factor is usually denoted as $\beta$. Can be thought about as how much of current consumption I give up for the promise of future consumption.

Write Bellman's equation for this problem.

>$$V(x_t) = max_{0 \le c \le x} \{u(c) + \beta V(x-c)\}$$
>
>Where $V(\cdot)$ represents a value function, which for each $c$ given $x$ it gives the maximized expected value of following the optimal plan, now and in the future. 

Guess that the optimal policy is to eat a constant fraction of the magic cake in every period. Assume that:
$$u(c) = \frac{c^{1-r} - 1}{1-r}$$
for $r \ge 0$. Fix $y_o z_0 = 10$, $r=2$, $\beta = 0.97$, $\mu = 0$, and $\sigma = 0.1$. Using np.random.seed(1234), generate a fixed sequence of 100 log normally distributed random variables. Use Python to search for the fraction that maximizes the expected reward for that fixed sequence.  You can work together.  Please indicate joint submissions clearly.  You will find a lot of code on Sargent's web page that will help you with aspects of this problem.

In [3]:
# Finding fraction that maximizes the expected reward for that fixed sequence
y_0_z_0 = 10 # initial value
r = 2 # reward curvature
beta = 0.97 # discount factor
mu = 0 # mu for use in generating log-normal sequence
sigma = 0.1 # the sigma used
T = 100 # time horizon

np.random.seed(1234)
sequence = np.random.lognormal(mu, sigma, 100)

In [4]:
sequence

array([1.04827244, 0.88772118, 1.15404215, 0.96921851, 0.93047611,
       1.09277059, 1.08976147, 0.93833115, 1.00157087, 0.79910055,
       1.12187745, 1.10428117, 1.10002446, 0.8169924 , 0.96714414,
       1.00021186, 1.04137853, 1.02933112, 1.14124049, 0.85668023,
       0.97993932, 0.93650826, 1.01953041, 1.05690401, 1.14089741,
       0.95415368, 1.06988954, 0.83384919, 0.98185577, 1.11170727,
       0.96099697, 1.03431954, 1.11044169, 1.11025956, 1.09021152,
       0.98786507, 1.01254939, 0.96823594, 1.08781106, 1.27010053,
       1.00764906, 0.94492984, 1.00362073, 0.81261506, 1.02508878,
       0.91419107, 0.98641366, 1.00183059, 1.07846787, 1.02176023,
       1.08773862, 0.86538481, 0.8691867 , 0.98995893, 0.94665151,
       0.98564212, 1.03603615, 0.996455  , 1.05820474, 1.16715117,
       0.90717162, 0.9929902 , 1.03127602, 0.97936598, 1.1089128 ,
       0.78659218, 1.22514642, 0.89202321, 1.02141441, 1.07301459,
       0.92446183, 1.0472901 , 1.07296176, 1.05374533, 0.91153

In [5]:
def felicity(c,r):
    """
    This function gives us our felicity given amount of consumtion, and r, reward.
    """
    u = (c**(1-r) - 1) / (1-r)
    return u

In [6]:
def bigU(c,beta,r,T):
    """
    c is a sequence (planned consumption)
    beta is the discount factor
    r is the reward curvature
    T is the time period
    """
    U = 0
    for t in range(T):
        eric = beta**(t)*felicity(c[t],r)
        U += eric

    return U

In [7]:
# Law of motion

def motion(z,share):
    """
    Where z equals a sequence of numbers, in this case representing random shocks to the size of the cake
    share represents the consumption share we choose for this problem
    """
    T = len(z) # time horizion
    c = [] # empty list
    y_0 = 10
    c_0 = share*y_0
    c.append(c_0)
    for t in range(T):
        y_1 = z[t] * (y_0 - c_0)
        c_1 = share*y_1
        c.append(c_1)
        y_0 = y_1
        c_0 = c_1
        
    return c

Getting an idea on the amount of utility we receive based on the consistent consumption share we choose.

In [8]:
c = motion(sequence, 0.5)
bigU(c, beta, r, T)

-8.206221105554276e+27

In [9]:
c = motion(sequence, 0.10)
bigU(c, beta, r, T)

-15222.37030164996

In [10]:
c = motion(sequence, 0.09)
bigU(c, beta, r, T)

-6672.007810396796

In [11]:
c = motion(sequence, 0.00000000000000001)
bigU(c, beta, r, T)

-2.8696304819261875e+17

Now creating a loop to find the maximum value of utility, given a range where we think that utility could be (0.001 to 0.10).

In [12]:
# can limit our search for the optimal utility here
X = np.linspace(0.001, 0.10, num = 10000)
U = []
for x in X:
    c = motion(sequence, x)
    u = bigU(c,beta,r,T)
    U.append(u)

# converting list to an array and finding the max value of utility and the associated level of consumption
U = np.array(U)
max_utility = np.max(U)
ind = np.argmax(U)
cons_share = X[ind]

In [13]:
print('Our maximum utility from the Cake problem is:',max_utility)
print('Our consistent consumption share is:', cons_share)

Our maximum utility from the Cake problem is: -227.20878858840354
Our consistent consumption share is: 0.02416831683168317


So our utility is negative, but it could be worse. Each day for 100 days, we eat roughly 2.4% of the magic cake (subsistence levels of eating, but beats starvation). 